In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from sklearn.utils import shuffle

In [2]:
path=[r'C:\Users\shuvo\Downloads\Compressed\CUHK_training_photo\photo',
       r'C:\Users\shuvo\Downloads\Compressed\CUHK_training_sketch\sketch']

In [3]:
global params
params = {'path' : path,
          'batch_size' : 4,
          'input_size': (250,200),   
          'lrn_rate': 0.0001, 
          'epochs': 22}

In [4]:
def get_file_paths(path):
    img_data=os.listdir(path)
    paths = [os.path.join(path, img) for img in img_data]
    return paths

In [5]:
def load_data(path):
#    print(path)
    img = cv2.imread(path, 0)
#    if img is None: print (path);pass
#    img = cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
   
    img = cv2.resize(img, 
                     (params['input_size'][1],params['input_size'][0]) , # <-- input_size tuple of format (h,w)
                     cv2.INTER_AREA)
    return np.expand_dims(img, -1)

In [6]:
def load_sketch(path):
    img = cv2.imread(path,0)
    #if img is None : return (False, None)
    img = cv2.resize(img, 
                     (200,250), 
                     cv2.INTER_AREA)
    return np.expand_dims(img, -1)

In [7]:
def Network(Input): #input : [Batch_size, 250, 200, 1]
    with tf.name_scope("Network"):
        conv1 = tf.layers.conv2d(Input, filters = 32, kernel_size = 3, strides = 1, padding='SAME', activation = tf.nn.relu, name = 'conv1')
        pool1 = tf.layers.max_pooling2d(conv1, pool_size = 2, strides = 1, padding='SAME', name = 'POOL1')
        print(conv1)
        print(pool1)
        
        #multilayer perception convulation layer
        conv2 = tf.layers.conv2d(pool1, filters = 16, kernel_size = 1, strides = 1, activation = tf.nn.relu, padding='SAME', name = 'conv2')
        conv3 = tf.layers.conv2d(conv2, filters = 16, kernel_size = 1, strides = 1, activation = tf.nn.relu, padding='SAME', name = 'conv3')
        conv4 = tf.layers.conv2d(conv3, filters = 32, kernel_size = 1, strides = 1, activation = tf.nn.relu, padding='SAME', name = 'conv4')
        
        #last convulation layer
        gensketch=tf.layers.conv2d(conv4, filters = 3, kernel_size = 3, strides = 1, activation = tf.nn.relu, padding='SAME', name = 'gensketch')
        print(gensketch)
    return gensketch
    

In [8]:
def loss_function(gensketch, sketch):
    with tf.variable_scope('L2_loss') as scope:
        loss = tf.reduce_mean(tf.square(sketch-gensketch))
    return loss

In [9]:
a = 88
b = 4
a//b

22

In [10]:
def FSSN():
    
    batchsize = params['batch_size']
    
    x = tf.placeholder(dtype = tf.float32, shape = [batchsize, 250, 200, 1])
    z = tf.placeholder(dtype = tf.float32, shape = [batchsize, 250, 200, 1])
    y = Network(x)
    loss = loss_function(y, z)
    optimiz = tf.train.AdadeltaOptimizer(params['lrn_rate']).minimize(loss)
    sess=tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tf.summary.scalar('Loss_Value',loss)
    real = tf.summary.image("Input", x, max_outputs=2)
    generated = tf.summary.image("Output", y, max_outputs=2)
    print('Setting up summary op...')
    summary_op = tf.summary.merge_all()
   
    print("Setting up Saver...") 
    saver = tf.train.Saver() 
    train_summary_writer = tf.summary.FileWriter('./log_dirfssn/train', sess.graph)
    test_summary_writer = tf.summary.FileWriter('./log_dirfssn/test', sess.graph)
    
    photo_paths =get_file_paths(path[0])
    sketch_paths = get_file_paths(path[1])
    
    dataset_size = len(photo_paths)
    n_batches = dataset_size//batchsize
    print("Dataset Size %d \nNo of Batches %d " %(dataset_size,n_batches))
    photo_batch_data, sketch_batch_data = [], []
    
    counter = 1
    for epoch in range(params['epochs']):

        for idx in range(n_batches): 
            
            photo_batch_paths = photo_paths[idx * batchsize: (idx+1) * batchsize]
            photo_batch_data = np.array([load_data(path) for path in photo_batch_paths])
#             print("Train Batch shape ", str(batch_data.shape)) 
            sketch_batch_paths = sketch_paths[idx * batchsize: (idx+1) * batchsize]
            sketch_batch_data = np.array([load_sketch(path) for path in sketch_batch_paths])
  
            _, train_loss, train_summary_str = sess.run([optimiz, loss, summary_op], 
                                                  feed_dict= {x: photo_batch_data , z: sketch_batch_data})
            train_summary_writer.add_summary(train_summary_str, counter)
            
            if idx % 10 == 0:
                print("Epoch %d \t Batch %d \t Loss %.4f"%(epoch, idx+1, train_loss))
            
            counter += 1

In [11]:
tf.reset_default_graph()
global sess 

sess = tf.Session()
FSSN()

Tensor("Network/conv1/Relu:0", shape=(4, 250, 200, 32), dtype=float32)
Tensor("Network/POOL1/MaxPool:0", shape=(4, 250, 200, 32), dtype=float32)
Tensor("Network/gensketch/Relu:0", shape=(4, 250, 200, 3), dtype=float32)
Setting up summary op...
Setting up Saver...
Dataset Size 88 
No of Batches 22 
Epoch 0 	 Batch 1 	 Loss 48861.7227
Epoch 0 	 Batch 11 	 Loss 52729.6211
Epoch 0 	 Batch 21 	 Loss 52084.8320
Epoch 1 	 Batch 1 	 Loss 48861.1484
Epoch 1 	 Batch 11 	 Loss 52729.0000
Epoch 1 	 Batch 21 	 Loss 52084.1641
Epoch 2 	 Batch 1 	 Loss 48860.5195
Epoch 2 	 Batch 11 	 Loss 52728.3867
Epoch 2 	 Batch 21 	 Loss 52083.5273
Epoch 3 	 Batch 1 	 Loss 48859.9219
Epoch 3 	 Batch 11 	 Loss 52727.7891
Epoch 3 	 Batch 21 	 Loss 52082.8398
Epoch 4 	 Batch 1 	 Loss 48859.3008
Epoch 4 	 Batch 11 	 Loss 52727.1836
Epoch 4 	 Batch 21 	 Loss 52082.1680
Epoch 5 	 Batch 1 	 Loss 48858.6445
Epoch 5 	 Batch 11 	 Loss 52726.5586
Epoch 5 	 Batch 21 	 Loss 52081.4492
Epoch 6 	 Batch 1 	 Loss 48858.0078
Epoch

In [12]:
photo_paths =get_file_paths(path[0])
sketch_paths = get_file_paths(path[1])
    
dataset_size = len(photo_paths)
n_batches = dataset_size/4

In [13]:
n_batches

22.0

In [14]:
dataset_size

88

In [15]:
photo_paths

['C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-005-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-006-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-007-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-008-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-009-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-010-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-011-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-012-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-013-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-014-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_training_photo\\photo\\f-015-01.jpg',
 'C:\\Users\\shuvo\\Downloads\\Compressed\\CUHK_traini